# Part 1: Simple DeepShift-Q

First off we'll go through my first attempt at implementing something like DeepShift-Q. Although it seems to work, it's a bit crude.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

import sys
import math
import numpy as np

use_cuda = True

device = torch.device("cpu" if use_cuda else "cpu")

n_epochs=5
lr=1.0
gamma=0.7
batch_size = 64
test_batch_size = 1000

kwargs = {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)


# Baseline
Let's establish a baseline for "normal" model performance. We'll train the pytorch example CNN on mnist.

In [2]:
class BaselineNet(nn.Module):
    def __init__(self):
        super(BaselineNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):        
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [3]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        readout = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item())
        sys.stdout.write(readout)
        sys.stdout.flush()
        sys.stdout.write('\r')
        
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [4]:
model = BaselineNet().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, 2 + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [29984/60000 (100%)]	Loss: 0.128866
Test set: Average loss: 0.0571, Accuracy: 9809/10000 (98%)

Train Epoch: 2 [29984/60000 (100%)]	Loss: 0.042776
Test set: Average loss: 0.0393, Accuracy: 9860/10000 (99%)

Train Epoch: 3 [29984/60000 (100%)]	Loss: 0.003029
Test set: Average loss: 0.0320, Accuracy: 9889/10000 (99%)

Train Epoch: 4 [29984/60000 (100%)]	Loss: 0.023198
Test set: Average loss: 0.0306, Accuracy: 9901/10000 (99%)

Train Epoch: 5 [29984/60000 (100%)]	Loss: 0.014392
Test set: Average loss: 0.0277, Accuracy: 9903/10000 (99%)



# Rounding to powers of 2

Now let's define some functions to go about quantizing weights to their nearest power of 2.

In [5]:
def get_shift_and_sign(x, rounding='deterministic'):
    # TODO: get the sign and the absolute value of x
    sign = np.sign(x)
    x_abs = np.abs(x)
    
    # TODO: now how do we get the shift parameter?
    shift = round(np.log2(x_abs), rounding)
    return (shift, sign)

def round(x, rounding='deterministic'):
    # we can experiment with stochastic or deterministic rounding
    assert(rounding in ['deterministic', 'stochastic'])
    if rounding == 'stochastic':
        x_floor = x.floor()
        return x_floor + torch.bernoulli(x - x_floor)
    else:
        return x.round()

def round_power_of_2(x, rounding='deterministic'):
    # TODO: we want this function to round x to the nearest 
    # power of 2
    shift, sign = get_shift_and_sign(x)
    
    # TODO: once we have the shift and sign, how do we get the
    # rounded version of x?
    x_rounded = (2 ** shift) * sign
    return x_rounded
    
def quantize_power_of_2(layer):
    # we'll use this function to directly modify the weights
    for param in layer.parameters():
        # TODO: use our rounding function to quantize the parameters
        param.data = round_power_of_2(param.data)

In [6]:
class CrudeNet(nn.Module):
    def __init__(self):
        super(CrudeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        

    def forward(self, x):
        # TODO: we'll need to call our quantize function here
        # which layers do we want to quantize?
        quantize_power_of_2(self.conv1)
        quantize_power_of_2(self.conv2)
        quantize_power_of_2(self.fc1)
        quantize_power_of_2(self.fc2)

        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output
    

In [7]:
model = CrudeNet().to(device)

# we'll use SGD for this one, since the authors claimed
# that SGD works the best for DeepShift-Q
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, 2 + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [29984/60000 (100%)]	Loss: 0.181032
Test set: Average loss: 0.1522, Accuracy: 9585/10000 (96%)

Train Epoch: 2 [29984/60000 (100%)]	Loss: 0.120429
Test set: Average loss: 0.1393, Accuracy: 9623/10000 (96%)

Train Epoch: 3 [29984/60000 (100%)]	Loss: 0.184143
Test set: Average loss: 0.1377, Accuracy: 9626/10000 (96%)

Train Epoch: 4 [29984/60000 (100%)]	Loss: 0.145351
Test set: Average loss: 0.1377, Accuracy: 9629/10000 (96%)



KeyboardInterrupt: 

# Question: 
Why are we able to get away with crudely quantizing the layers like this without explicitly messing with the gradient?

The only operation that isn't differentiable here is rounding and we don't take the gradient of that operation.


# Part 2: Implementing DeepShift-PS

It really does seem like our crude quantization works well enough, but the way it is implemented leaves a bad taste in my mouth. Now we'll implement DeepShift-PS in a fashion that is maybe a bit more satisfying. We will need to get a bit more clever to do this. Before starting, take a look at ste.py and answer the following question.

# Question:
How are the straight-through estimators implemented in ste.py? Why do these functions give us the desired behavior? (Hint: think back to when we looked at autograd) 

The STEs are wrapper layers that pass gradients backwards without applying any operations. This means that they preserve the gradient flow through the network, effectively setting their gradients with respect to their inputs to 1.


In [23]:
from torch.autograd import Function
from torch.nn.modules.utils import _pair
from torch.nn import init

import utils as utils
import ste as ste # ***if you're running in google colab, you'll need to manually import this to the runtime

# Implementing a bitshift linear layer

We need to make a custom pytorch layer to get the desired implementation. You might want to take a look at https://pytorch.org/docs/stable/notes/extending.html

In [24]:
class LinearShift(nn.Module):
    def __init__(self, in_features, out_features, 
                 bias=True, freeze_sign=False, use_cuda=True, 
                 rounding='deterministic', weight_bits=5):
        
        super(LinearShift, self).__init__()
        
        self.in_features = in_features
        self.out_features = out_features
        self.use_cuda = use_cuda
        self.rounding = rounding
        self.shift_range = (-1 * (2**(weight_bits - 1) - 2), 0) 
        
        # nn.Parameter is a special kind of Tensor, that will get
        # automatically registered as Module's parameter once it's assigned
        # as an attribute. Parameters and buffers need to be registered, or
        # they won't appear in .parameters() (doesn't apply to buffers), and
        # won't be converted when e.g. .cuda() is called. You can use
        # .register_buffer() to register buffers.
        # nn.Parameters require gradients by default.
        
        # here we register the shift (P) and sign (S) as parameters
        self.shift = nn.Parameter(torch.Tensor(out_features, in_features))
        self.sign = nn.Parameter(torch.Tensor(out_features, in_features), requires_grad = (freeze_sign == False))

        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_features))
        else:
            # You should always register all possible parameters, but the
            # optional ones can be None if you want.
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        self.shift.data.uniform_(*self.shift_range) 
        self.sign.data.uniform_(-1, 1) 
        
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.shift)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)

    def forward(self, input):
        # force P into a predefined range
        self.shift.data = ste.clamp(self.shift.data, *self.shift_range)
        
        # Hint: use the straight-through estimator functions for this part     
        # TODO: make a temporary rounded version of P
        shift_rounded = ste.round(self.shift, self.rounding)
        
        # TODO: make a temproary rounded version of S
        sign_rounded_signed = ste.sign(ste.round(self.sign))
        
        # TODO: make a temporary weight matrix built from P and S
        weight_ps = ste.unsym_grad_mul(2**shift_rounded, sign_rounded_signed)
    
        # with all that said and done, we can just pass W_ps to a vanilla
        # pytorch linear layer

        return torch.nn.functional.linear(input, weight_ps, self.bias)

# Implementing a bitshift convolutional layer

Now that we have our linear layer, we'll make a convolutional layer. This will be similar to what we just did.

In [25]:
class Conv2dShift(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size,
                 stride=1, padding=0, dilation=1,# output_padding,
                 groups=1, bias=True, padding_mode='zeros', 
                 rounding='deterministic', weight_bits=5):
        
        super(Conv2dShift, self).__init__()
        
        if in_channels % groups != 0:
            raise ValueError('in_channels must be divisible by groups')
        if out_channels % groups != 0:
            raise ValueError('out_channels must be divisible by groups')
        
        kernel_size = _pair(kernel_size)
        stride = _pair(stride)
        padding = _pair(padding)
        dilation = _pair(dilation)
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.groups = groups
        self.padding_mode = padding_mode
        self.rounding=rounding
        self.shift_range = (-1 * (2**(weight_bits - 1) - 2), 0)

        self.shift = nn.Parameter(torch.Tensor(
            out_channels, in_channels // groups, *kernel_size))
        self.sign = nn.Parameter(torch.Tensor(
            out_channels, in_channels // groups, *kernel_size),
            requires_grad = True)
        
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)
        
        self.reset_parameters()

    def reset_parameters(self):
        self.shift.data.uniform_(-10, -1) # (-0.1, 0.1)
        self.sign.data.uniform_(-1, 1) # (-0.1, 0.1)
        
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.shift)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)

    def forward(self, input):
        # force P into a predefined range
        self.shift.data = ste.clamp(self.shift.data, *self.shift_range)
        
        ## Hint: use the straight-through estimator functions for this part     
        # TODO: make a temporary rounded version of P
        shift_rounded = ste.round(self.shift)
        
        # TODO: make a temproary rounded version of S
        sign_rounded_signed = ste.sign(ste.round(self.sign))
        
        # TODO: make a temporary weight matrix built from P and S
        weight_ps = ste.unsym_grad_mul(2**shift_rounded, sign_rounded_signed)
        
        if self.padding_mode == 'circular':
            expanded_padding = ((self.padding[1] + 1) // 2, self.padding[1] // 2,
                                (self.padding[0] + 1) // 2, self.padding[0] // 2)

            input_padded = F.pad(input, expanded_padding, mode='circular')
            padding =  _pair(0)
        else:
            input_padded = input
            padding = self.padding
            
        return torch.nn.functional.conv2d(input_padded, weight_ps, self.bias, 
                                          self.stride, padding, self.dilation, self.groups)

In [26]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = Conv2dShift(1, 32, 3, 1)
        self.conv2 = Conv2dShift(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = LinearShift(9216, 128)
        self.fc2 = LinearShift(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        
        output = F.log_softmax(x, dim=1)
        return output

In [28]:
n_epochs=20

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, n_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [29984/60000 (100%)]	Loss: 0.900949
Test set: Average loss: 0.5475, Accuracy: 8623/10000 (86%)

Train Epoch: 2 [29984/60000 (100%)]	Loss: 1.030364
Test set: Average loss: 0.4169, Accuracy: 9029/10000 (90%)

Train Epoch: 3 [29984/60000 (100%)]	Loss: 0.463277
Test set: Average loss: 0.2892, Accuracy: 9203/10000 (92%)

Train Epoch: 4 [29984/60000 (100%)]	Loss: 0.588159
Test set: Average loss: 0.2987, Accuracy: 9226/10000 (92%)

Train Epoch: 5 [29984/60000 (100%)]	Loss: 0.900194
Test set: Average loss: 0.2729, Accuracy: 9176/10000 (92%)

Train Epoch: 6 [29984/60000 (100%)]	Loss: 0.818257
Test set: Average loss: 0.2397, Accuracy: 9306/10000 (93%)

Train Epoch: 7 [29984/60000 (100%)]	Loss: 0.426755
Test set: Average loss: 0.2035, Accuracy: 9409/10000 (94%)

Train Epoch: 8 [29984/60000 (100%)]	Loss: 0.415003
Test set: Average loss: 0.1996, Accuracy: 9420/10000 (94%)

Train Epoch: 9 [29984/60000 (100%)]	Loss: 0.521391
Test set: Average loss: 0.2045, Accuracy: 9453/10000 (95%)

T

# Final thoughts

I had a hard time getting comparable accuracy to the baseline with this, but play around with it and see what you can come up with.

I ended up getting around 95% accuring with Deepshift-PS.